# HDF5 (PyTables)

HDFStore is a dict-like object which reads and writes pandas using the high performance HDF5 format using the excellent PyTables library. See the cookbook for some advanced strategies

Warning: pandas requires PyTables >= 3.0.0. There is a indexing bug in PyTables < 3.2 which may appear when querying stores using an index. If you see a subset of results being returned, upgrade to PyTables >= 3.2. Stores created previously will need to be rewritten using the updated version.


In [54]:
import numpy as np
import pandas as pd
from datetime import timedelta

In [2]:
store = pd.HDFStore('store.h5')
print(store)

<class 'pandas.io.pytables.HDFStore'>
File path: store.h5



Objects can be written to the file just like adding key-value pairs to a dict:

In [3]:
np.random.seed(1234)

In [4]:
index = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])

In [5]:
data = np.random.randn(2, 5, 4).reshape(2, 20).T
wp = pd.DataFrame(data = data,
                  index=pd.MultiIndex.from_product([pd.date_range('1/1/2000', periods=5), ['A', 'B', 'C', 'D']]),
                  columns=['Item1', 'Item2'])
wp

Item1     Item2
2000-01-01 A  1.058969  0.215269
           B -0.397840  0.841009
           C  0.337438 -1.445810
           D  1.047579 -1.401973
2000-01-02 A  1.045938 -0.100918
           B  0.863717 -0.548242
           C -0.122092 -0.144620
           D  0.124713  0.354020
2000-01-03 A -0.322795 -0.035513
           B  0.841675  0.565738
           C  2.390961  1.545659
           D  0.076200 -0.974236
2000-01-04 A -0.566446 -0.070345
           B  0.036142  0.307969
           C -2.074978 -0.208499
           D  0.247792  1.033801
2000-01-05 A -0.897157 -2.400454
           B -0.136795  2.030604
           C  0.018289 -1.142631
           D  0.755414  0.211883

wp = pd.Panel(np.random.randn(2, 5, 4), items=['Item1', 'Item2'], 
              major_axis=pd.date_range('1/1/2000', periods=5), 
              minor_axis=['A', 'B', 'C', 'D']).to_frame()
wp

In [6]:
# store.put('s', s) is an equivalent method
store['s'] = s
store['df'] = df
store['wp'] = wp

In [7]:
# the type of stored data
store.root.wp._v_attrs.pandas_type

'frame'

In a current or later Python session, you can retrieve stored objects:

In [8]:
# store.get('df') is an equivalent method
store['df']

# dotted (attribute) access provides get as well
#store.df

,A,B,C
2000-01-01,0.887163,0.859588,-0.636524
2000-01-02,0.015696,-2.242685,1.150036
2000-01-03,0.991946,0.953324,-2.021255
2000-01-04,-0.334077,0.002118,0.405453
2000-01-05,0.289092,1.321158,-1.546906
2000-01-06,-0.202646,-0.655969,0.193421
2000-01-07,0.553439,1.318152,-0.469305
2000-01-08,0.675554,-1.817027,-0.183109


Deletion of the object specified by the key:

In [9]:
# store.remove('wp') is an equivalent method
del store['wp']

Closing a Store and using a context manager:

In [10]:
store.close()

In [11]:
store.is_open

False

In [12]:
# Working with, and automatically closing the store using a context manager
with pd.HDFStore('store.h5') as store:
    store.keys()

## Read/Write API

HDFStore supports an top-level API using read_hdf for reading and to_hdf for writing, similar to how read_csv and to_csv work.

In [13]:
df_tl = pd.DataFrame(dict(A=list(range(5)), B=list(range(5))))

In [14]:
df_tl.to_hdf('store_tl.h5','table', append=True)

In [15]:
pd.read_hdf('store_tl.h5', 'table', where=['index>2'])

,A,B
3,3,3
4,4,4


HDFStore will by default not drop rows that are all missing. This behavior can be changed by setting dropna=True.

In [16]:
df_with_missing = pd.DataFrame({'col1': [0, np.nan, 2], 
                                'col2': [1, np.nan, np.nan]})

In [17]:
df_with_missing

,col1,col2
0,0.0,1.0
1,NaN,NaN
2,2.0,NaN


In [18]:
df_with_missing.to_hdf('file.h5', 'df_with_missing', format='table', mode='w')

In [19]:
pd.read_hdf('file.h5', 'df_with_missing')

,col1,col2
0,0.0,1.0
1,NaN,NaN
2,2.0,NaN


In [20]:
df_with_missing.to_hdf('file.h5', 'df_with_missing', format='table', mode='w', dropna=True)

In [21]:
pd.read_hdf('file.h5', 'df_with_missing')

,col1,col2
0,0.0,1.0
2,2.0,NaN


This is also true for the major axis of a Panel:

In [22]:
data = np.array([[[np.nan, np.nan, np.nan], [1, np.nan, np.nan]], 
          [[np.nan, np.nan, np.nan], [np.nan, 5, 6]], 
          [[np.nan, np.nan, np.nan], [np.nan, 3, np.nan]]])
#data = np.random.randn(2, 5, 4)
panel_with_major_axis_all_missing = pd.DataFrame(data = data.reshape(3, 6).T,
                  index=pd.MultiIndex.from_product([[1, 2], ['A', 'B', 'C']]),
                  columns=['Item1', 'Item2', 'Item3'])
panel_with_major_axis_all_missing

Item1  Item2  Item3
1 A    NaN    NaN    NaN
  B    NaN    NaN    NaN
  C    NaN    NaN    NaN
2 A    1.0    NaN    NaN
  B    NaN    5.0    3.0
  C    NaN    6.0    NaN

In [23]:
panel_with_major_axis_all_missing.to_hdf('file.h5', 'panel', dropna=True, format='table', mode='w')

In [24]:
reloaded = pd.read_hdf('file.h5', 'panel')
reloaded

Item1  Item2  Item3
1 A    NaN    NaN    NaN
  B    NaN    NaN    NaN
  C    NaN    NaN    NaN
2 A    1.0    NaN    NaN
  B    NaN    5.0    3.0
  C    NaN    6.0    NaN

## Fixed Format

The examples above show storing using put, which write the HDF5 to PyTables in a fixed array format, called the fixed format. These types of stores are not appendable once written (though you can simply remove them and rewrite). Nor are they queryable; they must be retrieved in their entirety. They also do not support dataframes with non-unique column names. The fixed format stores offer very fast writing and slightly faster reading than table stores. This format is specified by default when using put or to_hdf or by format='fixed' or format='f'.

Warning : A fixed format will raise a TypeError if you try to retrieve using a where.

In [25]:
pd.DataFrame(np.random.randn(10, 2)).to_hdf('test_fixed.h5', 'df')
#pd.read_hdf('test_fixed.h5', 'df', where='index>5')

#TypeError: cannot pass a where specification when reading a fixed format.
#           this store must be selected in its entirety

## Table Format

HDFStore supports another PyTables format on disk, the table format. Conceptually a table is shaped very much like a DataFrame, with rows and columns. A table may be appended to in the same or other sessions. In addition, delete and query type operations are supported. This format is specified by format='table' or format='t' to append or put or to_hdf.

This format can be set as an option as well pd.set_option('io.hdf.default_format','table') to enable put/append/to_hdf to by default store in the table format.

In [26]:
store = pd.HDFStore('store.h5')
df1 = df[0:4]
df2 = df[4:]

# append data (creates a table automatically)
store.put('df', df1.append(df2), format='table')
#store.put('df', df2)

#store

# select the entire object
store.select('df')

,A,B,C
2000-01-01,0.887163,0.859588,-0.636524
2000-01-02,0.015696,-2.242685,1.150036
2000-01-03,0.991946,0.953324,-2.021255
2000-01-04,-0.334077,0.002118,0.405453
2000-01-05,0.289092,1.321158,-1.546906
2000-01-06,-0.202646,-0.655969,0.193421
2000-01-07,0.553439,1.318152,-0.469305
2000-01-08,0.675554,-1.817027,-0.183109


In [27]:
# the type of stored data
store.root.df._v_attrs.pandas_type

'frame_table'

Note : You can also create a table by passing format='table' or format='t' to a put operation.

## Hierarchical Keys

Keys to a store can be specified as a string. These can be in a hierarchical path-name like format (e.g. foo/bar/bah), which will generate a hierarchy of sub-stores (or Groups in PyTables parlance). Keys can be specified with out the leading ‘/’ and are always absolute (e.g. ‘foo’ refers to ‘/foo’). Removal operations can remove everything in the sub-store and below, so be careful.

In [28]:
store.put('foo/bar/bah', df)
store.append('food/orange', df)
store.append('food/apple',  df)

#store

# a list of keys are returned
store.keys()

['/df', '/s', '/food/apple', '/food/orange', '/foo/bar/bah']

In [29]:
# remove all nodes under this level
store.remove('food')
store.keys()

['/df', '/s', '/foo/bar/bah']

Warning: Hierarchical keys cannot be retrieved as dotted (attribute) access as described above for items stored under the root node.

In [30]:
# you can directly access the actual PyTables node but using the root node
store.root.foo.bar.bah

/foo/bar/bah (Group) ''
  children := ['axis0' (Array), 'axis1' (Array), 'block0_values' (Array), 'block0_items' (Array)]

Instead, use explicit string based keys:

In [31]:
store['foo/bar/bah']

,A,B,C
2000-01-01,0.887163,0.859588,-0.636524
2000-01-02,0.015696,-2.242685,1.150036
2000-01-03,0.991946,0.953324,-2.021255
2000-01-04,-0.334077,0.002118,0.405453
2000-01-05,0.289092,1.321158,-1.546906
2000-01-06,-0.202646,-0.655969,0.193421
2000-01-07,0.553439,1.318152,-0.469305
2000-01-08,0.675554,-1.817027,-0.183109


## Storing Types
### Storing Mixed Types in a Table

Storing mixed-dtype data is supported. Strings are stored as a fixed-width using the maximum size of the appended column. Subsequent attempts at appending longer strings will raise a ValueError.

Passing min_itemsize={`values`: size} as a parameter to append will set a larger minimum for the string columns. Storing floats, strings, ints, bools, datetime64 are currently supported. For string columns, passing nan_rep = 'nan' to append will change the default nan representation on disk (which converts to/from np.nan), this defaults to nan.

In [32]:
df_mixed = pd.DataFrame({'A': np.random.randn(8),
                         'B': np.random.randn(8),
                         'C': np.array(np.random.randn(8), dtype='float32'),
                         'string':'string',
                         'int': 1,
                         'bool': True,
                         'datetime64': pd.Timestamp('20010102')},
                        index=list(range(8)))

In [33]:
df_mixed.loc[df_mixed.index[3:5], ['A', 'B', 'string', 'datetime64']] = np.nan

In [34]:
store.append('df_mixed', df_mixed, min_itemsize = {'values': 50})

In [35]:
df_mixed1 = store.select('df_mixed')
df_mixed1

,A,B,C,string,int,bool,datetime64
0,1.391986,0.528288,1.363482,string,1,True,2001-01-02
1,0.079842,-1.068989,-0.781105,string,1,True,2001-01-02
2,-0.399965,-0.511881,-0.468018,string,1,True,2001-01-02
3,NaN,NaN,1.224574,NaN,1,True,NaT
4,NaN,NaN,-1.281108,NaN,1,True,NaT
5,0.816594,0.503592,0.875476,string,1,True,2001-01-02
6,-0.081947,0.285296,-1.710715,string,1,True,2001-01-02
7,-0.344766,0.484288,-0.450765,string,1,True,2001-01-02


In [36]:
df_mixed1.get_dtype_counts()

float64           2
float32           1
object            1
int64             1
bool              1
datetime64[ns]    1
dtype: int64

In [37]:
# we have provided a minimum string column size
store.root.df_mixed.table

/df_mixed/table (Table(8,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Float64Col(shape=(2,), dflt=0.0, pos=1),
  "values_block_1": Float32Col(shape=(1,), dflt=0.0, pos=2),
  "values_block_2": Int64Col(shape=(1,), dflt=0, pos=3),
  "values_block_3": Int64Col(shape=(1,), dflt=0, pos=4),
  "values_block_4": BoolCol(shape=(1,), dflt=False, pos=5),
  "values_block_5": StringCol(itemsize=50, shape=(1,), dflt=b'', pos=6)}
  byteorder := 'little'
  chunkshape := (689,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False}

## Storing Multi-Index DataFrames
Storing multi-index DataFrames as tables is very similar to storing/selecting from homogeneous index DataFrames.

In [38]:
index = pd.MultiIndex(levels=[['foo', 'bar', 'baz', 'qux'],
                              ['one', 'two', 'three']],
                      labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3, 3],
                              [0, 1, 2, 0, 1, 1, 2, 0, 1, 2]],
                      names=['foo', 'bar'])

In [39]:
df_mi = pd.DataFrame(np.random.randn(10, 3), index=index,
                     columns=['A', 'B', 'C'])
df_mi

A         B         C
foo bar                                
foo one    0.749164 -0.203933 -0.182175
    two    0.680656 -1.818499  0.047072
    three  0.394844 -0.248432 -0.617707
bar one   -0.682884  0.436258 -1.703013
    two    0.393711 -0.479324 -0.299016
baz two    0.694103  0.678630  0.239556
    three  0.151227  0.816127  1.893534
qux one    0.639633 -0.962029 -2.085266
    two    1.930247 -1.735349  1.210384
    three  0.797435 -0.379811  0.702562

In [40]:
store.append('df_mi', df_mi)

In [41]:
store.select('df_mi')

A         B         C
foo bar                                
foo one    0.749164 -0.203933 -0.182175
    two    0.680656 -1.818499  0.047072
    three  0.394844 -0.248432 -0.617707
bar one   -0.682884  0.436258 -1.703013
    two    0.393711 -0.479324 -0.299016
baz two    0.694103  0.678630  0.239556
    three  0.151227  0.816127  1.893534
qux one    0.639633 -0.962029 -2.085266
    two    1.930247 -1.735349  1.210384
    three  0.797435 -0.379811  0.702562

In [42]:
# the levels are automatically included as data columns
store.select('df_mi', 'foo=bar')

A         B         C
foo bar                              
bar one -0.682884  0.436258 -1.703013
    two  0.393711 -0.479324 -0.299016

## Querying
### Querying a Table

select and delete operations have an optional criterion that can be specified to select/delete only a subset of the data. This allows one to have a very large on-disk table and retrieve only a portion of the data.

A query is specified using the Term class under the hood, as a boolean expression.

    index and columns are supported indexers of a DataFrames.
    major_axis, minor_axis, and items are supported indexers of the Panel.
    if data_columns are specified, these can be used as additional indexers.

Valid comparison operators are:

=, ==, !=, >, >=, <, <=

Valid boolean expressions are combined with:

    | : or
    & : and
    ( and ) : for grouping

These rules are similar to how boolean expressions are used in pandas for indexing.

Note

    = will be automatically expanded to the comparison operator ==
    ~ is the not operator, but can only be used in very limited circumstances
    If a list/tuple of expressions is passed they will be combined via &

The following are valid expressions:

    'index >= date'
    "columns = ['A', 'D']"
    "columns in ['A', 'D']"
    'columns = A'
    'columns == A'
    "~(columns = ['A', 'B'])"
    'index > df.index[3] & string = "bar"'
    '(index > df.index[3] & index <= df.index[6]) | string = "bar"'
    "ts >= Timestamp('2012-02-01')"
    "major_axis>=20130101"

The indexers are on the left-hand side of the sub-expression:

columns, major_axis, ts

The right-hand side of the sub-expression (after a comparison operator) can be:

    functions that will be evaluated, e.g. Timestamp('2012-02-01')
    strings, e.g. "bar"
    date-like, e.g. 20130101, or "20130101"
    lists, e.g. "['A', 'B']"
    variables that are defined in the local names space, e.g. date

Note

Passing a string to a query by interpolating it into the query expression is not recommended. Simply assign the string of interest to a variable and use that variable in an expression. For example, do this

string = "HolyMoly'"
store.select('df', 'index == string')

instead of this

string = "HolyMoly'"
store.select('df',  'index == %s' % string)

The latter will not work and will raise a SyntaxError.Note that there’s a single quote followed by a double quote in the string variable.

If you must interpolate, use the '%r' format specifier

store.select('df', 'index == %r' % string)

which will quote string.

Here are some examples:

In [43]:
dfq = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'),
                   index=pd.date_range('20130101', periods=10))

In [44]:
store.append('dfq', dfq, format='table', data_columns=True)

Use boolean expressions, with in-line function evaluation.

In [45]:
store.select('dfq', "index>pd.Timestamp('20130104') & columns=['A', 'B']")

,A,B
2013-01-05,0.334198,0.536784
2013-01-06,-0.916199,-0.859668
2013-01-07,0.186494,0.952478
2013-01-08,-0.550603,-0.938153
2013-01-09,-0.223019,2.123692
2013-01-10,1.422986,-2.147855


Use and inline column reference

In [46]:
store.select('dfq', where="A>0 or C>0")

,A,B,C,D
2013-01-02,0.984188,-0.121728,2.365769,0.496143
2013-01-03,0.796595,-0.474021,-0.056696,1.357797
2013-01-05,0.334198,0.536784,-0.743830,-0.320204
2013-01-06,-0.916199,-0.859668,0.225985,0.628776
2013-01-07,0.186494,0.952478,0.988138,-0.072608
2013-01-09,-0.223019,2.123692,0.122273,-1.409432
2013-01-10,1.422986,-2.147855,-1.347533,0.363565


Works with a Panel as well.

In [47]:
store.append('wp', wp)
store

<class 'pandas.io.pytables.HDFStore'>
File path: store.h5

In [49]:
store.select('wp', "major_axis>pd.Timestamp('20000102') & minor_axis=['A', 'B']")

ValueError: The passed where expression: major_axis>pd.Timestamp('20000102') & minor_axis=['A', 'B']
            contains an invalid variable reference
            all of the variable references must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: index,columns,level_1,level_0


In [ ]:
Out[417]: 
<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 3 (major_axis) x 2 (minor_axis)
Items axis: Item1 to Item2
Major_axis axis: 2000-01-03 00:00:00 to 2000-01-05 00:00:00
Minor_axis axis: A to B

The columns keyword can be supplied to select a list of columns to be returned, this is equivalent to passing a 'columns=list_of_columns_to_filter':

In [50]:
store.select('df', "columns=['A', 'B']")

,A,B
2000-01-01,0.887163,0.859588
2000-01-02,0.015696,-2.242685
2000-01-03,0.991946,0.953324
2000-01-04,-0.334077,0.002118
2000-01-05,0.289092,1.321158
2000-01-06,-0.202646,-0.655969
2000-01-07,0.553439,1.318152
2000-01-08,0.675554,-1.817027


start and stop parameters can be specified to limit the total search space. These are in terms of the total number of rows in a table.

In [51]:
# this is effectively what the storage of a Panel looks like
wp.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [53]:
# limiting the search
store.select('wp', "major_axis>20000102 & minor_axis=['A', 'B']", start=0, stop=10)

ValueError: The passed where expression: major_axis>20000102 & minor_axis=['A', 'B']
            contains an invalid variable reference
            all of the variable references must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: index,columns,level_1,level_0


Note

select will raise a ValueError if the query expression has an unknown variable reference. Usually this means that you are trying to select on a column that is not a data_column.

select will raise a SyntaxError if the query expression is not valid.
Using timedelta64[ns]

You can store and query using the timedelta64[ns] type. Terms can be specified in the format: <float>(<unit>), where float may be signed (and fractional), and unit can be D,s,ms,us,ns for the timedelta. Here’s an example:

In [57]:
dftd = pd.DataFrame(dict(A = pd.Timestamp('20130101'), 
                         B = [ pd.Timestamp('20130101') + timedelta(days=i, seconds=10) for i in range(10) ]))
dftd

,A,B
0,2013-01-01,2013-01-01 00:00:10
1,2013-01-01,2013-01-02 00:00:10
2,2013-01-01,2013-01-03 00:00:10
3,2013-01-01,2013-01-04 00:00:10
4,2013-01-01,2013-01-05 00:00:10
5,2013-01-01,2013-01-06 00:00:10
6,2013-01-01,2013-01-07 00:00:10
7,2013-01-01,2013-01-08 00:00:10
8,2013-01-01,2013-01-09 00:00:10
9,2013-01-01,2013-01-10 00:00:10


In [56]:
dftd['C'] = dftd['A'] - dftd['B']
dftd

,A,B,C
0,2013-01-01,2013-01-01 00:00:10,-1 days +23:59:50
1,2013-01-01,2013-01-02 00:00:10,-2 days +23:59:50
2,2013-01-01,2013-01-03 00:00:10,-3 days +23:59:50
3,2013-01-01,2013-01-04 00:00:10,-4 days +23:59:50
4,2013-01-01,2013-01-05 00:00:10,-5 days +23:59:50
5,2013-01-01,2013-01-06 00:00:10,-6 days +23:59:50
6,2013-01-01,2013-01-07 00:00:10,-7 days +23:59:50
7,2013-01-01,2013-01-08 00:00:10,-8 days +23:59:50
8,2013-01-01,2013-01-09 00:00:10,-9 days +23:59:50
9,2013-01-01,2013-01-10 00:00:10,-10 days +23:59:50


In [58]:
store.append('dftd', dftd, data_columns=True)
store.select('dftd', "C<'-3.5D'")

ValueError: The passed where expression: C<'-3.5D'
            contains an invalid variable reference
            all of the variable references must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: index,columns,A,B


Indexing

You can create/modify an index for a table with create_table_index after data is already in the table (after and append/put operation). Creating a table index is highly encouraged. This will speed your queries a great deal when you use a select with the indexed dimension as the where.

Note

Indexes are automagically created on the indexables and any data columns you specify. This behavior can be turned off by passing index=False to append.

In [62]:
# we have automagically already created an index (in the first section)
i = store.root.df.table.cols.index.index
i.optlevel, i.kind

(9, 'full')

In [61]:
# change an index by passing new parameters
store.create_table_index('df', optlevel=9, kind='full')

In [63]:
i = store.root.df.table.cols.index.index
i.optlevel, i.kind

(9, 'full')

Often when appending large amounts of data to a store, it is useful to turn off index creation for each append, then recreate at the end.

In [65]:
df_1 = pd.DataFrame(np.random.randn(10, 2), columns=list('AB'))
df_2 = pd.DataFrame(np.random.randn(10, 2), columns=list('AB'))
st = pd.HDFStore('appends.h5', mode='w')
st.append('df', df_1, data_columns=['B'], index=False)
st.append('df', df_2, data_columns=['B'], index=False)
st.get_storer('df').table

/df/table (Table(20,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Float64Col(shape=(1,), dflt=0.0, pos=1),
  "B": Float64Col(shape=(), dflt=0.0, pos=2)}
  byteorder := 'little'
  chunkshape := (2730,)

Then create the index when finished appending.

In [66]:
st.create_table_index('df', columns=['B'], optlevel=9, kind='full')

In [67]:
st.get_storer('df').table

/df/table (Table(20,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Float64Col(shape=(1,), dflt=0.0, pos=1),
  "B": Float64Col(shape=(), dflt=0.0, pos=2)}
  byteorder := 'little'
  chunkshape := (2730,)
  autoindex := True
  colindexes := {
    "B": Index(9, full, shuffle, zlib(1)).is_csi=True}

In [68]:
st.close()

See here for how to create a completely-sorted-index (CSI) on an existing store.
Query via Data Columns

You can designate (and index) certain columns that you want to be able to perform queries (other than the indexable columns, which you can always query). For instance say you want to perform this common operation, on-disk, and return just the frame that matches this query. You can specify data_columns = True to force all columns to be data_columns.

In [69]:
df_dc = df.copy()
df_dc['string'] = 'foo'
df_dc.loc[df_dc.index[4: 6], 'string'] = np.nan
df_dc.loc[df_dc.index[7: 9], 'string'] = 'bar'
df_dc['string2'] = 'cool'
df_dc.loc[df_dc.index[1: 3], ['B', 'C']] = 1.0
df_dc

,A,B,C,string,string2
2000-01-01,0.887163,0.859588,-0.636524,foo,cool
2000-01-02,0.015696,1.000000,1.000000,foo,cool
2000-01-03,0.991946,1.000000,1.000000,foo,cool
2000-01-04,-0.334077,0.002118,0.405453,foo,cool
2000-01-05,0.289092,1.321158,-1.546906,NaN,cool
2000-01-06,-0.202646,-0.655969,0.193421,NaN,cool
2000-01-07,0.553439,1.318152,-0.469305,foo,cool
2000-01-08,0.675554,-1.817027,-0.183109,bar,cool


In [70]:
# on-disk operations
store.append('df_dc', df_dc, data_columns = ['B', 'C', 'string', 'string2'])

In [71]:
store.select('df_dc', where='B > 0')

,A,B,C,string,string2
2000-01-01,0.887163,0.859588,-0.636524,foo,cool
2000-01-02,0.015696,1.000000,1.000000,foo,cool
2000-01-03,0.991946,1.000000,1.000000,foo,cool
2000-01-04,-0.334077,0.002118,0.405453,foo,cool
2000-01-05,0.289092,1.321158,-1.546906,NaN,cool
2000-01-07,0.553439,1.318152,-0.469305,foo,cool


In [72]:
# getting creative
store.select('df_dc', 'B > 0 & C > 0 & string == foo')

,A,B,C,string,string2
2000-01-02,0.015696,1.000000,1.000000,foo,cool
2000-01-03,0.991946,1.000000,1.000000,foo,cool
2000-01-04,-0.334077,0.002118,0.405453,foo,cool


In [73]:
# this is in-memory version of this type of selection
df_dc[(df_dc.B > 0) & (df_dc.C > 0) & (df_dc.string == 'foo')]

,A,B,C,string,string2
2000-01-02,0.015696,1.000000,1.000000,foo,cool
2000-01-03,0.991946,1.000000,1.000000,foo,cool
2000-01-04,-0.334077,0.002118,0.405453,foo,cool


In [48]:
# we have automagically created this index and the B/C/string/string2
# columns are stored separately as ``PyTables`` columns
In [452]: store.root.df_dc.table
Out[452]: 
/df_dc/table (Table(8,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Float64Col(shape=(1,), dflt=0.0, pos=1),
  "B": Float64Col(shape=(), dflt=0.0, pos=2),
  "C": Float64Col(shape=(), dflt=0.0, pos=3),
  "string": StringCol(itemsize=3, shape=(), dflt=b'', pos=4),
  "string2": StringCol(itemsize=4, shape=(), dflt=b'', pos=5)}
  byteorder := 'little'
  chunkshape := (1680,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False,
    "B": Index(6, medium, shuffle, zlib(1)).is_csi=False,
    "C": Index(6, medium, shuffle, zlib(1)).is_csi=False,
    "string": Index(6, medium, shuffle, zlib(1)).is_csi=False,
    "string2": Index(6, medium, shuffle, zlib(1)).is_csi=False}

There is some performance degradation by making lots of columns into data columns, so it is up to the user to designate these. In addition, you cannot change data columns (nor indexables) after the first append/put operation (Of course you can simply read in the data and create a new table!).
Iterator

You can pass iterator=True or chunksize=number_in_a_chunk to select and select_as_multiple to return an iterator on the results. The default is 50,000 rows returned in a chunk.

In [453]: for df in store.select('df', chunksize=3):
   .....:    print(df)
   .....: 
                   A         B         C
2000-01-01  0.887163  0.859588 -0.636524
2000-01-02  0.015696 -2.242685  1.150036
2000-01-03  0.991946  0.953324 -2.021255
                   A         B         C
2000-01-04 -0.334077  0.002118  0.405453
2000-01-05  0.289092  1.321158 -1.546906
2000-01-06 -0.202646 -0.655969  0.193421
                   A         B         C
2000-01-07  0.553439  1.318152 -0.469305
2000-01-08  0.675554 -1.817027 -0.183109

Note

You can also use the iterator with read_hdf which will open, then automatically close the store when finished iterating.

for df in pd.read_hdf('store.h5','df', chunksize=3):
    print(df)

Note, that the chunksize keyword applies to the source rows. So if you are doing a query, then the chunksize will subdivide the total rows in the table and the query applied, returning an iterator on potentially unequal sized chunks.

Here is a recipe for generating a query and using it to create equal sized return chunks.

In [454]: dfeq = pd.DataFrame({'number': np.arange(1, 11)})

In [455]: dfeq
Out[455]: 
   number
0       1
1       2
2       3
3       4
4       5
5       6
6       7
7       8
8       9
9      10

In [456]: store.append('dfeq', dfeq, data_columns=['number'])

In [457]: def chunks(l, n):
   .....:      return [l[i: i+n] for i in range(0, len(l), n)]
   .....: 

In [458]: evens = [2, 4, 6, 8, 10]

In [459]: coordinates = store.select_as_coordinates('dfeq', 'number=evens')

In [460]: for c in chunks(coordinates, 2):
   .....:      print(store.select('dfeq', where=c))
   .....: 
   number
1       2
3       4
   number
5       6
7       8
   number
9      10

Advanced Queries
Select a Single Column

To retrieve a single indexable or data column, use the method select_column. This will, for example, enable you to get the index very quickly. These return a Series of the result, indexed by the row number. These do not currently accept the where selector.

In [461]: store.select_column('df_dc', 'index')
Out[461]: 
0   2000-01-01
1   2000-01-02
2   2000-01-03
3   2000-01-04
4   2000-01-05
5   2000-01-06
6   2000-01-07
7   2000-01-08
Name: index, dtype: datetime64[ns]

In [462]: store.select_column('df_dc', 'string')
Out[462]: 
0    foo
1    foo
2    foo
3    foo
4    NaN
5    NaN
6    foo
7    bar
Name: string, dtype: object

Selecting coordinates

Sometimes you want to get the coordinates (a.k.a the index locations) of your query. This returns an Int64Index of the resulting locations. These coordinates can also be passed to subsequent where operations.

In [463]: df_coord = pd.DataFrame(np.random.randn(1000, 2),
   .....:                         index=pd.date_range('20000101', periods=1000))
   .....: 

In [464]: store.append('df_coord', df_coord)

In [465]: c = store.select_as_coordinates('df_coord', 'index > 20020101')

In [466]: c
Out[466]: 
Int64Index([732, 733, 734, 735, 736, 737, 738, 739, 740, 741,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
           dtype='int64', length=268)

In [467]: store.select('df_coord', where=c)
Out[467]: 
                   0         1
2002-01-02 -0.178266 -0.064638
2002-01-03 -1.204956 -3.880898
2002-01-04  0.974470  0.415160
2002-01-05  1.751967  0.485011
2002-01-06 -0.170894  0.748870
2002-01-07  0.629793  0.811053
2002-01-08  2.133776  0.238459
...              ...       ...
2002-09-20 -0.181434  0.612399
2002-09-21 -0.763324 -0.354962
2002-09-22 -0.261776  0.812126
2002-09-23  0.482615 -0.886512
2002-09-24 -0.037757 -0.562953
2002-09-25  0.897706  0.383232
2002-09-26 -1.324806  1.139269

[268 rows x 2 columns]

Selecting using a where mask

Sometime your query can involve creating a list of rows to select. Usually this mask would be a resulting index from an indexing operation. This example selects the months of a datetimeindex which are 5.

In [468]: df_mask = pd.DataFrame(np.random.randn(1000, 2),
   .....:                        index=pd.date_range('20000101', periods=1000))
   .....: 

In [469]: store.append('df_mask', df_mask)

In [470]: c = store.select_column('df_mask', 'index')

In [471]: where = c[pd.DatetimeIndex(c).month == 5].index

In [472]: store.select('df_mask', where=where)
Out[472]: 
                   0         1
2000-05-01 -1.006245 -0.616759
2000-05-02  0.218940  0.717838
2000-05-03  0.013333  1.348060
2000-05-04  0.662176 -1.050645
2000-05-05 -1.034870 -0.243242
2000-05-06 -0.753366 -1.454329
2000-05-07 -1.022920 -0.476989
...              ...       ...
2002-05-25 -0.509090 -0.389376
2002-05-26  0.150674  1.164337
2002-05-27 -0.332944  0.115181
2002-05-28 -1.048127 -0.605733
2002-05-29  1.418754 -0.442835
2002-05-30 -0.433200  0.835001
2002-05-31 -1.041278  1.401811

[93 rows x 2 columns]

Storer Object

If you want to inspect the stored object, retrieve via get_storer. You could use this programmatically to say get the number of rows in an object.

In [473]: store.get_storer('df_dc').nrows
Out[473]: 8

Multiple Table Queries

The methods append_to_multiple and select_as_multiple can perform appending/selecting from multiple tables at once. The idea is to have one table (call it the selector table) that you index most/all of the columns, and perform your queries. The other table(s) are data tables with an index matching the selector table’s index. You can then perform a very fast query on the selector table, yet get lots of data back. This method is similar to having a very wide table, but enables more efficient queries.

The append_to_multiple method splits a given single DataFrame into multiple tables according to d, a dictionary that maps the table names to a list of ‘columns’ you want in that table. If None is used in place of a list, that table will have the remaining unspecified columns of the given DataFrame. The argument selector defines which table is the selector table (which you can make queries from). The argument dropna will drop rows from the input DataFrame to ensure tables are synchronized. This means that if a row for one of the tables being written to is entirely np.NaN, that row will be dropped from all tables.

If dropna is False, THE USER IS RESPONSIBLE FOR SYNCHRONIZING THE TABLES. Remember that entirely np.Nan rows are not written to the HDFStore, so if you choose to call dropna=False, some tables may have more rows than others, and therefore select_as_multiple may not work or it may return unexpected results.

In [474]: df_mt = pd.DataFrame(randn(8, 6), index=pd.date_range('1/1/2000', periods=8),
   .....:                                   columns=['A', 'B', 'C', 'D', 'E', 'F'])
   .....: 

In [475]: df_mt['foo'] = 'bar'

In [476]: df_mt.loc[df_mt.index[1], ('A', 'B')] = np.nan

# you can also create the tables individually
In [477]: store.append_to_multiple({'df1_mt': ['A', 'B'], 'df2_mt': None },
   .....:                           df_mt, selector='df1_mt')
   .....: 

In [478]: store
Out[478]: 
<class 'pandas.io.pytables.HDFStore'>
File path: store.h5

# individual tables were created
In [479]: store.select('df1_mt')
Out[479]: 
                   A         B
2000-01-01  0.714697  0.318215
2000-01-02       NaN       NaN
2000-01-03 -0.086919  0.416905
2000-01-04  0.489131 -0.253340
2000-01-05 -0.382952 -0.397373
2000-01-06  0.538116  0.226388
2000-01-07 -2.073479 -0.115926
2000-01-08 -0.695400  0.402493

In [480]: store.select('df2_mt')
Out[480]: 
                   C         D         E         F  foo
2000-01-01  0.607460  0.790907  0.852225  0.096696  bar
2000-01-02  0.811031 -0.356817  1.047085  0.664705  bar
2000-01-03 -0.764381 -0.287229 -0.089351 -1.035115  bar
2000-01-04 -1.948100 -0.116556  0.800597 -0.796154  bar
2000-01-05 -0.717627  0.156995 -0.344718 -0.171208  bar
2000-01-06  1.541729  0.205256  1.998065  0.953591  bar
2000-01-07  1.391070  0.303013  1.093347 -0.101000  bar
2000-01-08 -1.507639  0.089575  0.658822 -1.037627  bar

# as a multiple
In [481]: store.select_as_multiple(['df1_mt', 'df2_mt'], where=['A>0', 'B>0'],
   .....:                           selector = 'df1_mt')
   .....: 
Out[481]: 
                   A         B         C         D         E         F  foo
2000-01-01  0.714697  0.318215  0.607460  0.790907  0.852225  0.096696  bar
2000-01-06  0.538116  0.226388  1.541729  0.205256  1.998065  0.953591  bar

Delete from a Table

You can delete from a table selectively by specifying a where. In deleting rows, it is important to understand the PyTables deletes rows by erasing the rows, then moving the following data. Thus deleting can potentially be a very expensive operation depending on the orientation of your data. This is especially true in higher dimensional objects (Panel and Panel4D). To get optimal performance, it’s worthwhile to have the dimension you are deleting be the first of the indexables.

Data is ordered (on the disk) in terms of the indexables. Here’s a simple use case. You store panel-type data, with dates in the major_axis and ids in the minor_axis. The data is then interleaved like this:

    date_1 - id_1 - id_2 - . - id_n
    date_2 - id_1 - . - id_n

It should be clear that a delete operation on the major_axis will be fairly quick, as one chunk is removed, then the following data moved. On the other hand a delete operation on the minor_axis will be very expensive. In this case it would almost certainly be faster to rewrite the table using a where that selects all but the missing data.

# returns the number of rows deleted
In [482]: store.remove('wp', 'major_axis > 20000102' )
Out[482]: 12

In [483]: store.select('wp')
Out[483]: 
<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 2 (major_axis) x 4 (minor_axis)
Items axis: Item1 to Item2
Major_axis axis: 2000-01-01 00:00:00 to 2000-01-02 00:00:00
Minor_axis axis: A to D

Warning

Please note that HDF5 DOES NOT RECLAIM SPACE in the h5 files automatically. Thus, repeatedly deleting (or removing nodes) and adding again, WILL TEND TO INCREASE THE FILE SIZE.

To repack and clean the file, use ptrepack.
Notes & Caveats
Compression

PyTables allows the stored data to be compressed. This applies to all kinds of stores, not just tables. Two parameters are used to control compression: complevel and complib.

complevel specifies if and how hard data is to be compressed.
    complevel=0 and complevel=None disables compression and 0<complevel<10 enables compression.
complib specifies which compression library to use. If nothing is

    specified the default library zlib is used. A compression library usually optimizes for either good compression rates or speed and the results will depend on the type of data. Which type of compression to choose depends on your specific needs and data. The list of supported compression libraries:

            zlib: The default compression library. A classic in terms of compression, achieves good compression rates but is somewhat slow.
            lzo: Fast compression and decompression.
            bzip2: Good compression rates.
            blosc: Fast compression and decompression.

        New in version 0.20.2: Support for alternative blosc compressors:

            blosc:blosclz This is the default compressor for blosc
            blosc:lz4: A compact, very popular and fast compressor.
            blosc:lz4hc: A tweaked version of LZ4, produces better compression ratios at the expense of speed.
            blosc:snappy: A popular compressor used in many places.
            blosc:zlib: A classic; somewhat slower than the previous ones, but achieving better compression ratios.
            blosc:zstd: An extremely well balanced codec; it provides the best compression ratios among the others above, and at reasonably fast speed.

        If complib is defined as something other than the listed libraries a ValueError exception is issued.

Note

If the library specified with the complib option is missing on your platform, compression defaults to zlib without further ado.

Enable compression for all objects within the file:

store_compressed = pd.HDFStore('store_compressed.h5', complevel=9,
                               complib='blosc:blosclz')

Or on-the-fly compression (this only applies to tables) in stores where compression is not enabled:

store.append('df', df, complib='zlib', complevel=5)

ptrepack

PyTables offers better write performance when tables are compressed after they are written, as opposed to turning on compression at the very beginning. You can use the supplied PyTables utility ptrepack. In addition, ptrepack can change compression levels after the fact.

ptrepack --chunkshape=auto --propindexes --complevel=9 --complib=blosc in.h5 out.h5

Furthermore ptrepack in.h5 out.h5 will repack the file to allow you to reuse previously deleted space. Alternatively, one can simply remove the file and write again, or use the copy method.
Caveats

Warning

HDFStore is not-threadsafe for writing. The underlying PyTables only supports concurrent reads (via threading or processes). If you need reading and writing at the same time, you need to serialize these operations in a single thread in a single process. You will corrupt your data otherwise. See the (GH2397) for more information.

    If you use locks to manage write access between multiple processes, you may want to use fsync() before releasing write locks. For convenience you can use store.flush(fsync=True) to do this for you.
    Once a table is created its items (Panel) / columns (DataFrame) are fixed; only exactly the same columns can be appended
    Be aware that timezones (e.g., pytz.timezone('US/Eastern')) are not necessarily equal across timezone versions. So if data is localized to a specific timezone in the HDFStore using one version of a timezone library and that data is updated with another version, the data will be converted to UTC since these timezones are not considered equal. Either use the same version of timezone library or use tz_convert with the updated timezone definition.

Warning

PyTables will show a NaturalNameWarning if a column name cannot be used as an attribute selector. Natural identifiers contain only letters, numbers, and underscores, and may not begin with a number. Other identifiers cannot be used in a where clause and are generally a bad idea.
DataTypes

HDFStore will map an object dtype to the PyTables underlying dtype. This means the following types are known to work:
Type 	Represents missing values
floating : float64, float32, float16 	np.nan
integer : int64, int32, int8, uint64,uint32, uint8 	 
boolean 	 
datetime64[ns] 	NaT
timedelta64[ns] 	NaT
categorical : see the section below 	 
object : strings 	np.nan

unicode columns are not supported, and WILL FAIL.
Categorical Data

You can write data that contains category dtypes to a HDFStore. Queries work the same as if it was an object array. However, the category dtyped data is stored in a more efficient manner.

In [484]: dfcat = pd.DataFrame({'A': pd.Series(list('aabbcdba')).astype('category'),
   .....:                       'B': np.random.randn(8) })
   .....: 

In [485]: dfcat
Out[485]: 
   A         B
0  a  0.603273
1  a  0.262554
2  b -0.979586
3  b  2.132387
4  c  0.892485
5  d  1.996474
6  b  0.231425
7  a  0.980070

In [486]: dfcat.dtypes
Out[486]: 
A    category
B     float64
dtype: object

In [487]: cstore = pd.HDFStore('cats.h5', mode='w')

In [488]: cstore.append('dfcat', dfcat, format='table', data_columns=['A'])

In [489]: result = cstore.select('dfcat', where="A in ['b', 'c']")

In [490]: result
Out[490]: 
   A         B
2  b -0.979586
3  b  2.132387
4  c  0.892485
6  b  0.231425

In [491]: result.dtypes
Out[491]: 
A    category
B     float64
dtype: object

String Columns

min_itemsize

The underlying implementation of HDFStore uses a fixed column width (itemsize) for string columns. A string column itemsize is calculated as the maximum of the length of data (for that column) that is passed to the HDFStore, in the first append. Subsequent appends, may introduce a string for a column larger than the column can hold, an Exception will be raised (otherwise you could have a silent truncation of these columns, leading to loss of information). In the future we may relax this and allow a user-specified truncation to occur.

Pass min_itemsize on the first table creation to a-priori specify the minimum length of a particular string column. min_itemsize can be an integer, or a dict mapping a column name to an integer. You can pass values as a key to allow all indexables or data_columns to have this min_itemsize.

Passing a min_itemsize dict will cause all passed columns to be created as data_columns automatically.

Note

If you are not passing any data_columns, then the min_itemsize will be the maximum of the length of any string passed

In [492]: dfs = pd.DataFrame(dict(A='foo', B='bar'), index=list(range(5)))

In [493]: dfs
Out[493]: 
     A    B
0  foo  bar
1  foo  bar
2  foo  bar
3  foo  bar
4  foo  bar

# A and B have a size of 30
In [494]: store.append('dfs', dfs, min_itemsize=30)

In [495]: store.get_storer('dfs').table
Out[495]: 
/dfs/table (Table(5,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": StringCol(itemsize=30, shape=(2,), dflt=b'', pos=1)}
  byteorder := 'little'
  chunkshape := (963,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False}

# A is created as a data_column with a size of 30
# B is size is calculated
In [496]: store.append('dfs2', dfs, min_itemsize={'A': 30})

In [497]: store.get_storer('dfs2').table
Out[497]: 
/dfs2/table (Table(5,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": StringCol(itemsize=3, shape=(1,), dflt=b'', pos=1),
  "A": StringCol(itemsize=30, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (1598,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False,
    "A": Index(6, medium, shuffle, zlib(1)).is_csi=False}

nan_rep

String columns will serialize a np.nan (a missing value) with the nan_rep string representation. This defaults to the string value nan. You could inadvertently turn an actual nan value into a missing value.

In [498]: dfss = pd.DataFrame(dict(A=['foo', 'bar', 'nan']))

In [499]: dfss
Out[499]: 
     A
0  foo
1  bar
2  nan

In [500]: store.append('dfss', dfss)

In [501]: store.select('dfss')
Out[501]: 
     A
0  foo
1  bar
2  NaN

# here you need to specify a different nan rep
In [502]: store.append('dfss2', dfss, nan_rep='_nan_')

In [503]: store.select('dfss2')
Out[503]: 
     A
0  foo
1  bar
2  nan

External Compatibility

HDFStore writes table format objects in specific formats suitable for producing loss-less round trips to pandas objects. For external compatibility, HDFStore can read native PyTables format tables.

It is possible to write an HDFStore object that can easily be imported into R using the rhdf5 library (Package website). Create a table format store like this:

In [504]: np.random.seed(1)

In [505]: df_for_r = pd.DataFrame({"first": np.random.rand(100),
   .....:                          "second": np.random.rand(100),
   .....:                          "class": np.random.randint(0, 2, (100, ))},
   .....:                          index=range(100))
   .....: 

In [506]: df_for_r.head()
Out[506]: 
      first    second  class
0  0.417022  0.326645      0
1  0.720324  0.527058      0
2  0.000114  0.885942      1
3  0.302333  0.357270      1
4  0.146756  0.908535      1

In [507]: store_export = pd.HDFStore('export.h5')

In [508]: store_export.append('df_for_r', df_for_r, data_columns=df_dc.columns)

In [509]: store_export
Out[509]: 
<class 'pandas.io.pytables.HDFStore'>
File path: export.h5

In R this file can be read into a data.frame object using the rhdf5 library. The following example function reads the corresponding column names and data values from the values and assembles them into a data.frame:

# Load values and column names for all datasets from corresponding nodes and
# insert them into one data.frame object.

library(rhdf5)

loadhdf5data <- function(h5File) {

listing <- h5ls(h5File)
# Find all data nodes, values are stored in *_values and corresponding column
# titles in *_items
data_nodes <- grep("_values", listing$name)
name_nodes <- grep("_items", listing$name)
data_paths = paste(listing$group[data_nodes], listing$name[data_nodes], sep = "/")
name_paths = paste(listing$group[name_nodes], listing$name[name_nodes], sep = "/")
columns = list()
for (idx in seq(data_paths)) {
  # NOTE: matrices returned by h5read have to be transposed to obtain
  # required Fortran order!
  data <- data.frame(t(h5read(h5File, data_paths[idx])))
  names <- t(h5read(h5File, name_paths[idx]))
  entry <- data.frame(data)
  colnames(entry) <- names
  columns <- append(columns, entry)
}

data <- data.frame(columns)

return(data)
}

Now you can import the DataFrame into R:

> data = loadhdf5data("transfer.hdf5")
> head(data)
         first    second class
1 0.4170220047 0.3266449     0
2 0.7203244934 0.5270581     0
3 0.0001143748 0.8859421     1
4 0.3023325726 0.3572698     1
5 0.1467558908 0.9085352     1
6 0.0923385948 0.6233601     1




SyntaxError: invalid syntax (<ipython-input-48-30f9ec52b35e>, line 3)

Note

The R function lists the entire HDF5 file’s contents and assembles the data.frame object from all matching nodes, so use this only as a starting point if you have stored multiple DataFrame objects to a single HDF5 file.
Performance

    tables format come with a writing performance penalty as compared to fixed stores. The benefit is the ability to append/delete and query (potentially very large amounts of data). Write times are generally longer as compared with regular stores. Query times can be quite fast, especially on an indexed axis.
    You can pass chunksize=<int> to append, specifying the write chunksize (default is 50000). This will significantly lower your memory usage on writing.
    You can pass expectedrows=<int> to the first append, to set the TOTAL number of expected rows that PyTables will expected. This will optimize read/write performance.
    Duplicate rows can be written to tables, but are filtered out in selection (with the last items being selected; thus a table is unique on major, minor pairs)
    A PerformanceWarning will be raised if you are attempting to store types that will be pickled by PyTables (rather than stored as endemic types). See Here for more information and some solutions.
